# Re-scaling buildings to Household Units

In [1]:
%load_ext jupyter_black
import pandas as pd
import os
from pathlib import Path

In [2]:
base_url = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/"
)
input_dir = base_url / "input/Google Footprint Data/"

In [3]:
phl_ggl_bld_municip_count = pd.read_csv(
    input_dir / "phl_google_bld_municip_count.csv"
)
phl_ggl_bld_grid_count = pd.read_csv(
    input_dir / "phl_google_bld_grid_count.csv"
)
phl_ggl_bld_intersection_count = pd.read_csv(
    input_dir / "phl_google_bld_intersection_count.csv"
)

### Computing weights

#### Municipality to Grid

In [4]:
mun_to_grid = phl_ggl_bld_intersection_count.merge(
    phl_ggl_bld_municip_count, on="ADM3_PCODE", suffixes=("_x", None)
)
mun_to_grid["weight"] = (
    mun_to_grid["numbuildings_x"] / mun_to_grid["numbuildings"]
)
mun_to_grid[mun_to_grid["Centroid"] == "122.1E_17.4N"]

,ADM3_PCODE,ADM3_EN_x,id,Centroid,numbuildings_x,ADM3_EN,numbuildings,weight
7439,PH023106000,Cabagan,13064.0,122.1E_17.4N,0,Cabagan,26094,0.000000
7473,PH023133000,San Pablo,13064.0,122.1E_17.4N,0,San Pablo,13510,0.000000
7488,PH023137000,Tumauini,13064.0,122.1E_17.4N,0,Tumauini,35325,0.000000
7494,PH023117000,Maconacon,13064.0,122.1E_17.4N,1,Maconacon,1578,0.000634


In [5]:
mun_to_grid.to_csv(input_dir / "ggl_mun_to_grid_weights.csv", index=False)

#### Grid to Municipality

In [5]:
grid_to_mun = phl_ggl_bld_intersection_count.merge(
    phl_ggl_bld_grid_count, on="Centroid", suffixes=("_x", None)
)
grid_to_mun["weight"] = (
    grid_to_mun["numbuildings_x"] / grid_to_mun["numbuildings"]
)
grid_to_mun.groupby("Centroid").sum().sort_values(by="weight")

C:\Users\pauni\AppData\Local\Temp\ipykernel_14668\1780964883.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grid_to_mun.groupby("Centroid").sum().sort_values(by="weight")


,id_x,numbuildings_x,id,numbuildings,weight
Centroid,,,,,
114.3E_11.1N,101.0,0,101,1,0.0
120.3E_11.2N,10120.0,0,10120,1,0.0
120.3E_12.4N,10108.0,0,10108,1,0.0
120.3E_13.9N,10093.0,0,10093,1,0.0
122.0E_21.1N,12860.0,0,12860,1,0.0
...,...,...,...,...,...
121.6E_18.3N,48880.0,6037,48880,24148,1.0
121.6E_18.4N,24438.0,5661,24438,11322,1.0
121.6E_5.9N,12344.0,2,12344,2,1.0


In [7]:
grid_to_mun.to_csv(input_dir / "ggl_grid_to_mun_weights.csv", index=False)

### Re-scaling number of buildings with Housing Units data